# Auswertung der umgesetzten Projektlängen

Dieses Skript lädt alle Planungen aus der API herunter und wertet diese dann nach Bezirk, bzw. Jahr aus, so dass die Längen aller abgeschlossenen Planungen addiert werden, die in die jeweilige Kategorie fallen.

In [99]:
import requests
from collections import defaultdict

In [100]:
ENDPOINT = "https://fixmyberlin.de/api/v1/projects?page_size=200"
raw = requests.get(ENDPOINT).json()
results = raw["results"]
print("Got", len(results))

Got 187


In [101]:
results_2019 = [
    planning for planning in results 
    if planning['phase'] == "ready" 
]
len(results_2019)
results_2019 = results

In [102]:
def getRVALength(planning):
    isBisided = planning["side"] == 2
    return planning["length"] if not isBisided else planning["length"] * 2

In [107]:
by_borough = defaultdict(lambda: {"2016": 0, "2017": 0, "2018": 0, "2019": 0, "2020": 0, "2021": 0, "total": 0})
by_year = defaultdict(int)
for result in results_2019:
    try:
        bor = result["borough"]
        year = result["construction_completed_date"][:4]
    except TypeError:
        continue
    
    by_borough[bor][year] += getRVALength(result)
    by_borough[bor]["total"] += getRVALength(result)
    by_year[year] += getRVALength(result)
    
    if result["borough"] is None:
        print("Missing borough", result["url"])


In [127]:
out = f"Bezirk,Gesamt," + ",".join(sorted(by_borough['Mitte'].keys()[:-1])) + "\n"
for bor in sorted(by_borough.keys())[:-1]:
    total = by_borough[bor]["total"]
    years = ""
    for year in sorted(by_borough[bor].keys())[:-1]:
        # print(year, by_borough[bor][year])
        years += "," + str(by_borough[bor][year])
    out += f"{bor},{total}{years}\n"
print(out)

Bezirk,Gesamt,2016,2017,2018,2019,2020,2021,total
Friedrichshain-Kreuzberg,14463.0,0,0,642.0,853.0,12968.0,0
Lichtenberg,1050.0,0,0,1050.0,0,0,0
Marzahn-Hellersdorf,6696.0,0,0,3810.0,2886.0,0,0
Mitte,22896.0,0,0,684.0,15758.0,6454.0,0
Neukölln,14634.0,320.0,2650.0,5054.0,1548.0,3882.0,1180.0
Pankow,11904.0,0,2106.0,2744.0,6418.0,636.0,0
Spandau,1522.0,0,0,0,1522.0,0,0
Steglitz-Zehlendorf,21175.0,0,0,4830.0,16345.0,0,0
Tempelhof-Schöneberg,1140.0,0,0,0,1140.0,0,0



In [128]:
by_year

defaultdict(int,
            {'2020': 23940.0,
             '2019': 52880.0,
             '2018': 22753.0,
             '2021': 3592.0,
             '2017': 14418.0,
             '2016': 320.0})

In [129]:
with open('out.csv', "w") as f:
    f.write(out)